In [ ]:
import re
import ast
from django.db.models import Q

In [ ]:
import pandas as pd

In [ ]:
file = "../rita_data_ibk/vfbr.csv"

In [ ]:
df = pd.read_csv(file)

In [ ]:
repo, _ = Institution.objects.get_or_create(written_name="Südtiroler Landesarchiv", abbreviation="SLA")

In [ ]:
for i, row in df.iterrows():
    vfb_sig = row['Verfachbuch'].replace('-TEI-P5.xm', '')[:-1]
    vfb_sig = re.sub(r'(\s\[.*\])', '', vfb_sig)
    vfb_sig = re.sub(r'(,,)', ',', vfb_sig)
    vfb_sig = re.sub(r'(\(!\))', '', vfb_sig)
    vfb, _ = VerfachBuch.objects.get_or_create(
        signatur=vfb_sig
    )
    vfb.year = f"{row['start_year']}-01-01"
    vfb.year_latest = f"{row['end_year']}-01-01"
    vfb.year_label = row['years']
    vfb.repo = repo
    vfb.save()
    entry_sig = f"{vfb_sig}, {row['Eintrag'].split(':')[0].strip()}"
    entry_sig =  (re.sub(r'(\[.*\])', '', entry_sig))[:60]
    vfb_entry, _ = VfbEntry.objects.get_or_create(
        entry_signatur=entry_sig
    )
    vfb_entry.located_in = vfb
    vfb_entry.start_page = row['start']
    vfb_entry.end_page = row['end']
    vfb_entry.vollregest = row['clean_text']
    vfb_entry.save()
    for x in ast.literal_eval(row['notes']):
        note, _ = Anmerkungen.objects.get_or_create(
            text=x
        )
        note.entry = vfb_entry
        note.save()

In [ ]:
for x in VfbEntry.objects.filter(entry_signatur__startswith='VFB_LG_S'):
    new_sig = x.entry_signatur.replace('VFB_LG_StMichaelsburgl', 'VB. Lg. St. Michaelsburg')
    x.entry_signatur = new_sig
    x.save()

In [ ]:
for x in VerfachBuch.objects.filter(signatur__startswith='VFB_LG_S'):
    new_sig = x.signatur.replace('VFB_LG_StMichaelsburgl', 'VB. Lg. St. Michaelsburg')
    x.signatur = new_sig
    x.save()

In [ ]:
for x in VerfachBuch.objects.filter(signatur__startswith='VFB_STG_Bruneckl'):
    new_sig = x.signatur.replace('VFB_STG_Bruneckl', 'VB. Stdg. Bruneck')
    x.signatur = new_sig
    x.save()

In [ ]:
for x in VfbEntry.objects.filter(entry_signatur__startswith='VFB_STG_Bruneckl'):
    new_sig = x.entry_signatur.replace('VFB_STG_Bruneckl', 'VB. Stdg. Bruneck')
    x.entry_signatur = new_sig
    x.save()

In [ ]:
for x in VerfachBuch.objects.filter(signatur__startswith='VFB_OAG_Bruneckl'):
    new_sig = x.signatur.replace('VFB_OAG_Bruneckl', 'VB. Oberamtsgerich Bruneck')
    x.signatur = new_sig
    x.save()

In [ ]:
for x in VfbEntry.objects.filter(entry_signatur__startswith='VFB_OAG_Bruneckl'):
    new_sig = x.entry_signatur.replace('VFB_OAG_Bruneckl', 'VB. Oberamtsgerich Bruneck')
    x.entry_signatur = new_sig
    x.save()

In [ ]:
for x in Anmerkungen.objects.filter(Q(text__contains="BUCH") | Q(text__contains="BÜCHER")):
    entry = x.entry
    entry.book = True
    entry.save()

In [ ]:
for x in VfbEntry.objects.exclude(book=True):
    x.book = False
    x.save()

In [ ]:
for x in VfbEntry.objects.all():
    new = x.entry_signatur.replace('VB.', 'SLA, VB.')
    x.entry_signatur = new
    x.save()